In [ ]:
!pip install gensim scikit-learn

In [ ]:
import json
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef

In [ ]:
def preprocess_code(code_text):
    """Tiền xử lý mã nguồn Rust (đầu vào là chuỗi)."""
    code_text = re.sub(r'//.*?\n|/\*.*?\*/', '', code_text, flags=re.DOTALL)  # Xóa comments
    code_text = re.sub(r'[^\w\s]', '', code_text)  # Xóa ký tự đặc biệt
    code_text = code_text.lower()  # Chuyển về chữ thường
    return code_text.split()  # Tách thành các từ

In [ ]:
def train_word2vec(code_snippets):
    """Huấn luyện mô hình word2vec."""
    sentences = [preprocess_code(snippet) for snippet in code_snippets]
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    return model

In [ ]:
def get_feature_vector(code_json, model):
    """Trích xuất vector đặc trưng."""
    words = preprocess_code(code_json)
    vectors = [model.wv[word] for word in words if word in model.wv]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [ ]:
def get_feature_vector(code_json, model):
    """Trích xuất vector đặc trưng."""
    words = preprocess_code(code_json)
    vectors = [model.wv[word] for word in words if word in model.wv]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [ ]:
def prepare_data(json_file_path):
    """Chuẩn bị dữ liệu huấn luyện."""
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    code_snippets = [item['func'] for item in data]
    labels = [item['target'] for item in data]

    model = train_word2vec(code_snippets)
    features = [get_feature_vector(snippet, model) for snippet in code_snippets]

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test

In [ ]:
def train_logistic_regression(X_train, y_train, max_iter=100):  # Thêm max_iter
    """Huấn luyện mô hình logistic regression với giới hạn số vòng lặp."""
    model = LogisticRegression(max_iter=max_iter)  # Thiết lập max_iter
    model.fit(X_train, y_train)
    print("Số vòng lặp đã chạy:", model.n_iter_[0])  # In ra số vòng lặp
    return model

def evaluate_model(model, X_test, y_test):
    """Đánh giá mô hình."""
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Xác suất dự đoán cho class 1

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    pr_auc = average_precision_score(y_test, y_prob)
    mcc = matthews_corrcoef(y_test, y_pred)
    error_rate = 1 - accuracy

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("ROC AUC:", roc_auc)
    print("Precision-Recall AUC:", pr_auc)
    print("MCC:", mcc)
    print("Error Rate:", error_rate)

In [ ]:
# Sử dụng
json_file_path = "/content/dataset.rust.json"  # Thay thế bằng đường dẫn đến file JSON của bạn
max_iterations = 100  # Giới hạn số vòng lặp tối đa

X_train, X_test, y_train, y_test = prepare_data(json_file_path)
model = train_logistic_regression(X_train, y_train, max_iter=max_iterations)  # Truyền max_iter
evaluate_model(model, X_test, y_test)

Số vòng lặp đã chạy: 33
Accuracy: 0.7321428571428571
Precision: 0.7307692307692307
Recall: 0.7037037037037037
F1-score: 0.7169811320754716
ROC AUC: 0.8071519795657727
Precision-Recall AUC: 0.7400311365109602
MCC: 0.46321262548101694
Error Rate: 0.2678571428571429
